In [ ]:
import io
import os
import json

import country_converter as coco
import pandas as pd

from dfpp.storage import StorageManager
from dfpp.storage.postgres import AsyncPGClient

cc = coco.CountryConverter()

In [ ]:
async with StorageManager() as storage_manager:
    source_file_name = os.path.join(
        storage_manager.sources_path,
        f"{source_cfg['id'].upper()}.{source_cfg['file_format']}",
    )
    bytes_data = await storage_manager.read_blob(path=source_file_name)

source_data = bytes_data.decode("utf-8")
source_data_dict = json.loads(source_data)
df = pd.DataFrame(source_data_dict["value"])

In [ ]:
assert df.shape[0] > 0

In [ ]:
df.info()

In [ ]:
df.head()

In [ ]:
df.columns

In [ ]:
df["TimeDimType"].value_counts(dropna=False)

In [ ]:
df["TimeDimensionValue"].value_counts(dropna=False)

In [ ]:
df["IndicatorCode"].value_counts(dropna=False).shape[0]

In [ ]:
assert df["IndicatorCode"].value_counts(dropna=False).shape[0] == 1

In [ ]:
df["SpatialDimType"].value_counts()

In [ ]:
df_source = df.copy()

In [ ]:
df_source = df_source.loc[(df_source["TimeDimType"] == "YEAR")].reset_index(drop=True)

In [ ]:
df_source["Dim1Type"].value_counts(dropna=False)

In [ ]:
assert source_cfg["country_iso3_column"] in df_source
assert source_cfg["datetime_column"] in df_source

In [ ]:
assert (
    "IndicatorCode" in df_source.columns and df_source["IndicatorCode"].nunique() == 1
)

In [ ]:
df_source.rename(
    columns={
        source_cfg["datetime_column"]: "year",
        source_cfg["country_iso3_column"]: "alpha_3_code",
        "Value": indicator_cfg["indicator_id"],
    },
    inplace=True,
)

In [ ]:
indicator_cfg["group_name"]

In [ ]:
df_source.columns

In [ ]:
if (
    indicator_cfg["group_name"]
    and indicator_cfg["value_column"] in df_source["Dim1"].unique()
):
    df_source = df_source.loc[df_source["Dim1"] == indicator_cfg["value_column"]]

In [ ]:
df_source["country_or_area"] = cc.pandas_convert(
    series=df_source["alpha_3_code"], src="ISO3", to="name_official"
)

In [ ]:
df_source["alpha_3_code"].value_counts(dropna=True)

In [ ]:
df_source = df_source[
    ["alpha_3_code", "country_or_area", "year", indicator_cfg["indicator_id"]]
]

In [ ]:
df_source.head()

,alpha_3_code,country_or_area,employementoutsidetheformalsectorbysexandstatusinemploymentthousandsfemalenotclassified_ilopiflste
11,AFG,Islamic Republic of Afghanistan,9.943
41,AGO,Republic of Angola,147.283
74,AGO,Republic of Angola,94.062
107,ARG,Argentine Republic,NaN
305,ARM,Republic of Armenia,NaN


In [ ]:
async with AsyncPGClient() as pg_client:
    await pg_client.insert_indicator(indicator_cfg["indicator_id"], df_source)

Mock connected to the database.
Inserted 712 rows into table for indicator: informalemploymentbysexandstatusinemploymentthousandsfemaleemployees_iloniflste
Mock closed the database connection.
